## 1- Definition of arguments for function usage

In [1]:
import sys
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
from utils import *
import argparse
sys.argv = ['']

parser = argparse.ArgumentParser(description='Parameters training')
parser.add_argument('--model_architecture', type=str, default="VGG16", help='....')
parser.add_argument('--dataset', type=str, default="CIFAR10", help='....')
parser.add_argument('--batch_size', type=int, default=8, help='....')
parser.add_argument('--num_epochs', type=int, default=40, help='....')
parser.add_argument('--learning_rate', type=float, default=1e-3, help='....')
parser.add_argument('--optimizer_val', type=str, default="SGD", help='....')
parser.add_argument('--model_type', type=str, default="UNPRUNED", help='....')
parser.add_argument('--device', type=str, default=None, help='....')
parser.add_argument('--model_input', default=torch.ones((1, 3, 224, 224)), help='....')
parser.add_argument('--eval_metric', default="accuracy", help='....')
parser.add_argument('--pruning_seed', type=int, default=23, help='....')
parser.add_argument('--list_pruning', type=list, default = [0.6,0.6,0.53,0.53,0.4,0.4,0.4,0.5,0.5,0.5,0.6,0.6,0.6,0.5,0.5,0], help='....')

args = parser.parse_args()

if args.device is None:
    import torch
    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Modify for training and pruning
#args.model_architecture = "ResNet18"
#args.num_epochs = 40
#args.dataset = "Tomato_Leaves_noBg"
#args.eval_metric = "f1_score"

# Modify for pruning
args.list_pruning = [0.15,0.35,0.4,0.4,0.5,0.5,0.5,0.7,0.7,0.7,0.7,0.7,0.7,0.5,0.5,0]
type_pruning = 'CRECIENTE'
base_percentage = 50

## Get Model, DATASET and TRAIN

In [3]:
train_loader, test_loader, num_classes = get_dataset(args)

In [10]:
model = get_model(num_classes, args)

In [11]:
train_model(train_loader = train_loader,
            test_loader = test_loader,
            model = model,
            num_classes = num_classes,
            args = args)

Epoch: [1/40]	 || Training Loss: 0.365	 || Val Loss: 0.184	 || Training F1-score: 0.850 	 ||  Val F1-score: 0.916 	 || Time: 00:07:10
Model Name: VGG16_Tomato_Leaves_noBg_UNPRUNED
Epoch: [2/40]	 || Training Loss: 0.110	 || Val Loss: 0.062	 || Training F1-score: 0.958 	 ||  Val F1-score: 0.971 	 || Time: 00:14:33
Model Name: VGG16_Tomato_Leaves_noBg_UNPRUNED
Epoch: [3/40]	 || Training Loss: 0.060	 || Val Loss: 0.062	 || Training F1-score: 0.975 	 ||  Val F1-score: 0.972 	 || Time: 00:22:07
Model Name: VGG16_Tomato_Leaves_noBg_UNPRUNED
Epoch: [4/40]	 || Training Loss: 0.041	 || Val Loss: 0.047	 || Training F1-score: 0.983 	 ||  Val F1-score: 0.980 	 || Time: 00:29:38
Model Name: VGG16_Tomato_Leaves_noBg_UNPRUNED
Epoch: [5/40]	 || Training Loss: 0.027	 || Val Loss: 0.038	 || Training F1-score: 0.990 	 ||  Val F1-score: 0.987 	 || Time: 00:37:16
Model Name: VGG16_Tomato_Leaves_noBg_UNPRUNED
Epoch: [6/40]	 || Training Loss: 0.022	 || Val Loss: 0.044	 || Training F1-score: 0.991 	 ||  Val F1

## Pruning with multiple seeds

In [4]:
list_seeds = [23,42,97,112,167,214,256,333,425,512]

original_model_name = f'{args.model_architecture}_{args.dataset}_UNPRUNED'
for seed in list_seeds:
    #load original model
    model = torch.load(f'models/{original_model_name}.pth')
    args.seed = seed
    args.model_type = f'{type_pruning}_{base_percentage}_PRUNED_SEED_{seed}'
    #prune original model
    prune_model(model, args)
    args.model_type = f'{type_pruning}_{base_percentage}_PRUNED_FT_SEED_{seed}'
    #retraining pruned model
    train_model(train_loader = train_loader,
                test_loader = test_loader,
                model = model,
                num_classes = num_classes,
                args = args)
    print('============================')

Epoch: [1/40]	 || Training Loss: 0.885	 || Val Loss: 0.312	 || Training F1-score: 0.626 	 ||  Val F1-score: 0.865 	 || Time: 00:03:12
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_23
Epoch: [2/40]	 || Training Loss: 0.319	 || Val Loss: 0.163	 || Training F1-score: 0.863 	 ||  Val F1-score: 0.940 	 || Time: 00:06:23
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_23
Epoch: [3/40]	 || Training Loss: 0.211	 || Val Loss: 0.151	 || Training F1-score: 0.912 	 ||  Val F1-score: 0.932 	 || Time: 00:09:36
Epoch: [4/40]	 || Training Loss: 0.153	 || Val Loss: 0.114	 || Training F1-score: 0.937 	 ||  Val F1-score: 0.951 	 || Time: 00:12:48
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_23
Epoch: [5/40]	 || Training Loss: 0.114	 || Val Loss: 0.183	 || Training F1-score: 0.954 	 ||  Val F1-score: 0.917 	 || Time: 00:16:00
Epoch: [6/40]	 || Training Loss: 0.096	 || Val Loss: 0.127	 || Training F1-score: 0.960 	 ||  Val F1-score: 0.952 	 || Time: 0

Epoch: [14/40]	 || Training Loss: 0.027	 || Val Loss: 0.090	 || Training F1-score: 0.989 	 ||  Val F1-score: 0.965 	 || Time: 00:51:01
Epoch: [15/40]	 || Training Loss: 0.024	 || Val Loss: 0.179	 || Training F1-score: 0.990 	 ||  Val F1-score: 0.942 	 || Time: 00:54:40
Epoch: [16/40]	 || Training Loss: 0.018	 || Val Loss: 0.075	 || Training F1-score: 0.993 	 ||  Val F1-score: 0.971 	 || Time: 00:58:13
Epoch: [17/40]	 || Training Loss: 0.013	 || Val Loss: 0.054	 || Training F1-score: 0.995 	 ||  Val F1-score: 0.979 	 || Time: 01:02:00
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_42
Epoch: [18/40]	 || Training Loss: 0.022	 || Val Loss: 0.113	 || Training F1-score: 0.991 	 ||  Val F1-score: 0.964 	 || Time: 01:05:40
Epoch: [19/40]	 || Training Loss: 0.013	 || Val Loss: 0.065	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.979 	 || Time: 01:09:20
Epoch: [20/40]	 || Training Loss: 0.023	 || Val Loss: 0.064	 || Training F1-score: 0.991 	 ||  Val F1-score: 0.978 	 || Ti

Epoch: [28/40]	 || Training Loss: 0.015	 || Val Loss: 0.076	 || Training F1-score: 0.994 	 ||  Val F1-score: 0.977 	 || Time: 01:42:32
Epoch: [29/40]	 || Training Loss: 0.004	 || Val Loss: 0.047	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.983 	 || Time: 01:45:57
Epoch: [30/40]	 || Training Loss: 0.011	 || Val Loss: 0.065	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.980 	 || Time: 01:49:38
Epoch: [31/40]	 || Training Loss: 0.008	 || Val Loss: 0.055	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.976 	 || Time: 01:53:19
Epoch: [32/40]	 || Training Loss: 0.012	 || Val Loss: 0.109	 || Training F1-score: 0.995 	 ||  Val F1-score: 0.972 	 || Time: 01:56:57
Epoch: [33/40]	 || Training Loss: 0.014	 || Val Loss: 0.044	 || Training F1-score: 0.994 	 ||  Val F1-score: 0.985 	 || Time: 02:00:26
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_97
Epoch: [34/40]	 || Training Loss: 0.005	 || Val Loss: 0.047	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.984 	 || Ti

Epoch: [2/40]	 || Training Loss: 0.394	 || Val Loss: 0.270	 || Training F1-score: 0.826 	 ||  Val F1-score: 0.891 	 || Time: 00:07:17
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_167
Epoch: [3/40]	 || Training Loss: 0.242	 || Val Loss: 0.212	 || Training F1-score: 0.891 	 ||  Val F1-score: 0.913 	 || Time: 00:10:50
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_167
Epoch: [4/40]	 || Training Loss: 0.172	 || Val Loss: 0.205	 || Training F1-score: 0.929 	 ||  Val F1-score: 0.910 	 || Time: 00:14:32
Epoch: [5/40]	 || Training Loss: 0.132	 || Val Loss: 0.101	 || Training F1-score: 0.945 	 ||  Val F1-score: 0.957 	 || Time: 00:18:20
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_167
Epoch: [6/40]	 || Training Loss: 0.100	 || Val Loss: 0.088	 || Training F1-score: 0.959 	 ||  Val F1-score: 0.962 	 || Time: 00:21:58
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_167
Epoch: [7/40]	 || Training Loss: 0.078	 || Val Loss: 0

Epoch: [14/40]	 || Training Loss: 0.027	 || Val Loss: 0.088	 || Training F1-score: 0.989 	 ||  Val F1-score: 0.971 	 || Time: 00:51:51
Epoch: [15/40]	 || Training Loss: 0.020	 || Val Loss: 0.066	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.975 	 || Time: 00:55:33
Epoch: [16/40]	 || Training Loss: 0.017	 || Val Loss: 0.064	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.974 	 || Time: 00:59:09
Epoch: [17/40]	 || Training Loss: 0.023	 || Val Loss: 0.075	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.971 	 || Time: 01:02:51
Epoch: [18/40]	 || Training Loss: 0.027	 || Val Loss: 0.089	 || Training F1-score: 0.989 	 ||  Val F1-score: 0.966 	 || Time: 01:06:36
Epoch: [19/40]	 || Training Loss: 0.012	 || Val Loss: 0.095	 || Training F1-score: 0.995 	 ||  Val F1-score: 0.968 	 || Time: 01:10:31
Epoch: [20/40]	 || Training Loss: 0.013	 || Val Loss: 0.057	 || Training F1-score: 0.994 	 ||  Val F1-score: 0.977 	 || Time: 01:14:06
Epoch: [21/40]	 || Training Loss: 0.007	 || Val Loss: 0

Epoch: [29/40]	 || Training Loss: 0.009	 || Val Loss: 0.122	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.958 	 || Time: 01:48:28
Epoch: [30/40]	 || Training Loss: 0.016	 || Val Loss: 0.063	 || Training F1-score: 0.993 	 ||  Val F1-score: 0.980 	 || Time: 01:52:15
Epoch: [31/40]	 || Training Loss: 0.005	 || Val Loss: 0.045	 || Training F1-score: 0.999 	 ||  Val F1-score: 0.983 	 || Time: 01:56:03
Epoch: [32/40]	 || Training Loss: 0.002	 || Val Loss: 0.037	 || Training F1-score: 0.999 	 ||  Val F1-score: 0.986 	 || Time: 01:59:41
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_256
Epoch: [33/40]	 || Training Loss: 0.006	 || Val Loss: 0.041	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.987 	 || Time: 02:03:27
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_256
Epoch: [34/40]	 || Training Loss: 0.001	 || Val Loss: 0.045	 || Training F1-score: 1.000 	 ||  Val F1-score: 0.984 	 || Time: 02:07:12
Epoch: [35/40]	 || Training Loss: 0.010	 || Val Loss

Epoch: [4/40]	 || Training Loss: 0.152	 || Val Loss: 0.123	 || Training F1-score: 0.936 	 ||  Val F1-score: 0.946 	 || Time: 00:15:03
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_425
Epoch: [5/40]	 || Training Loss: 0.119	 || Val Loss: 0.114	 || Training F1-score: 0.948 	 ||  Val F1-score: 0.958 	 || Time: 00:18:56
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_425
Epoch: [6/40]	 || Training Loss: 0.100	 || Val Loss: 0.078	 || Training F1-score: 0.957 	 ||  Val F1-score: 0.965 	 || Time: 00:22:41
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_425
Epoch: [7/40]	 || Training Loss: 0.074	 || Val Loss: 0.105	 || Training F1-score: 0.967 	 ||  Val F1-score: 0.954 	 || Time: 00:26:33
Epoch: [8/40]	 || Training Loss: 0.056	 || Val Loss: 0.073	 || Training F1-score: 0.975 	 ||  Val F1-score: 0.971 	 || Time: 00:30:10
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_425
Epoch: [9/40]	 || Training Loss: 0.053	 || Val Loss: 0

Epoch: [15/40]	 || Training Loss: 0.032	 || Val Loss: 0.083	 || Training F1-score: 0.987 	 ||  Val F1-score: 0.965 	 || Time: 00:56:04
Epoch: [16/40]	 || Training Loss: 0.013	 || Val Loss: 0.060	 || Training F1-score: 0.995 	 ||  Val F1-score: 0.978 	 || Time: 00:59:44
Model Name: VGG16_Tomato_Leaves_noBg_CRECIENTE_50_PRUNED_FT_SEED_512
Epoch: [17/40]	 || Training Loss: 0.006	 || Val Loss: 0.067	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.976 	 || Time: 01:03:29
Epoch: [18/40]	 || Training Loss: 0.028	 || Val Loss: 0.084	 || Training F1-score: 0.988 	 ||  Val F1-score: 0.968 	 || Time: 01:07:18
Epoch: [19/40]	 || Training Loss: 0.019	 || Val Loss: 0.076	 || Training F1-score: 0.991 	 ||  Val F1-score: 0.974 	 || Time: 01:11:03
Epoch: [20/40]	 || Training Loss: 0.022	 || Val Loss: 0.075	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.973 	 || Time: 01:14:36
Epoch: [21/40]	 || Training Loss: 0.010	 || Val Loss: 0.077	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.970 	 || T